# FUA simplification with [`cityseer`](https://github.com/benchmark-urbanism/cityseer-api)

* **Simons, G.** (2023). *The cityseer Python package for pedestrian-scale network-based urban analysis*. Environment and Planning B: Urban Analytics and City Science, 50(5), 1328-1344. https://doi.org/10.1177/23998083221133827

In [1]:
%load_ext watermark
%watermark

Last updated: 2024-12-04T13:20:25.358364+01:00

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.26.0

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
import pathlib
import time

import cityseer
import geopandas

from core import utils

%watermark -w
%watermark -iv

Watermark: 2.4.3

geopandas: 1.0.1
core     : 0.1.dev155+ge86f29a.d20240823
cityseer : 4.15.1



In [6]:
def run_cityseer(city: str) -> geopandas.GeoDataFrame:
    """Run the ``cityseer`` workflow as laid out in
    [https://benchmark-urbanism.github.io/cityseer-examples/examples/graph_cleaning.html#manual-cleaning]
    with ***all default arguments***.

    Parameters
    ----------
    city : str
        City anme. See ``utils.city_fua.keys()``.

    Returns
    -------
    geopandas.GeoDataFrame
        Resultant simplified network edges – only geometry column.
        Will be written to ``.parquet``.
    """
    edges = utils.read_original(city)
    graph = cityseer.tools.io.nx_from_generic_geopandas(edges)
    graph = cityseer.tools.graphs.nx_remove_filler_nodes(graph)
    graph = cityseer.tools.graphs.nx_remove_dangling_nodes(graph)
    graph = cityseer.tools.graphs.nx_split_opposing_geoms(graph)
    graph = cityseer.tools.graphs.nx_remove_filler_nodes(graph)
    graph = cityseer.tools.graphs.nx_consolidate_nodes(graph)
    graph = cityseer.tools.graphs.nx_remove_filler_nodes(graph)
    graph = cityseer.tools.graphs.nx_iron_edges(graph)
    return cityseer.tools.io.geopandas_from_nx(graph, crs=edges.crs).rename_geometry(
        "geometry"
    )[["geometry"]]

In [10]:
utils.city_fua

{'Aleppo': 1133,
 'Auckland': 869,
 'Bucaramanga': 4617,
 'Douala': 809,
 'Liège': 1656,
 'Salt Lake City': 4881,
 'Wuhan': 8989}

In [4]:
output_base = pathlib.Path("..", "..", "data")

In [ ]:
global_start = time.time()

for city, fua in utils.city_fua.items():
    print("===================================================================")
    print(f"Starting {city}")
    print("-------------------------")
    print("\n\n")

    fua_start = time.time()
    simplified = run_cityseer(city)
    output_fua = output_base / pathlib.Path(f"{fua}", "cityseer", f"{fua}.parquet")
    simplified.to_parquet(output_fua)
    fua_end = time.time()
    fua_duration = round((fua_end - fua_start) / 60.0, 3)

    print(f"\t{city} completed in: {fua_duration} minutes")
    print("\n\n\n\n")

print("~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ")
print(" ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~")
print("~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ")
print(" ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~")

global_end = time.time()
global_duration = round((global_end - global_start) / 60.0, 3)

print(f"Total runtime: {global_duration} minutes")

Starting Salt Lake City
-------------------------





100%|██████████| 50917/50917 [00:03<00:00, 16610.66it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|██████████| 50917/50917 [00:00<00:00, 176495.40it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 46988/46988 [00:02<00:00, 17447.32it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████| 9539/9539 [00:00<00:00, 528184.74it/s]
INFO:cityseer.tools.util:Creating edges STR tree.
100%|██████████| 13400/13400 [00:00<00:00, 1228576.16it/s]
INFO:cityseer.tools.graphs:Splitting opposing edges.
100%|██████████| 9473/9473 [00:00<00:00, 17056.95it/s]
INFO:cityseer.tools.graphs:Squashing opposing nodes
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.
100%|██████████| 13571/13571 [00:00<00:00, 94405.58it/s] 
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 9473/9473 [00:00<00:00, 255589.35it/s]
INFO:cityseer.tools.util:Creating nodes STR tree
100%|██████████| 9246/9246 [00:00<00:00, 211729

	Salt Lake City completed in: 0.172 minutes





~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
Total runtime: 0.172 minutes
